In [1]:
import pandas as pd
import numpy as np
import re
import os

df = pd.read_json('../Data/Serie_A_2017_games.json')

df_squad = pd.read_json('../Data/Serie_A_2018_squads.json').T

df_transposto = df.transpose()

df_transposto.head(9)

,Home,Away,Result,Players,Goals,Changes,Yellow cards,Red cards
1,Fluminense / RJ,Santos / SP,3 X 2,[[12Diego Cava ... Diego Cavalieri T(g)P137990...,[03:00 1T9NRJose Henrique da Silva Dourado Flu...,[17:00 2TFluminense/RJ 5 - Lucas Pierre Santos...,"[09:00 2T9Ricardo de Oliveira Santos/SP, 16:00...",[]
2,Flamengo / RJ,Atlético Mineiro / MG,1 X 1,[[38Alex Muralha Alex Roberto Santana ... T(g)...,[23:00 1T42NRMatheus Gonçalves Savio Flamengo/...,[45:00 INTAtlético/MG 10 - Juan Ramon Cazares ...,"[14:00 2T15Ralph Machado Dias Atlético/MG, 20:...",[]
3,Palmeiras / SP,Vasco da Gama / RJ,4 X 0,[[1Fernando P ... Fernando Buttenbende ... T(g...,[40:00 1T18NRAlejandro Abraham Guerra Morales ...,[12:00 2TVasco da Gama/RJ 19 - Paulo Vitor Fer...,[32:00 2T3Jomar Herculano Lourenco Vasco da Ga...,[]
4,Corinthians / SP,Chapecoense / SC,1 X 1,"[[12Cassio Cassio Ramos T(g)P166276, Corinthia...",[21:00 1T7NRJoao Alves de Assis Silva Corinthi...,[43:00 1TCorinthians/SP 34 - Pedro Henrique Ri...,[17:00 2T23Fagner Conserva Lemos Corinthians/S...,[]
5,Cruzeiro / MG,São Paulo / SP,1 X 0,"[[1Fabio Fabio Deivson Lopes ... T(g)P129292,...",[02:00 2T9NRRamon Dario Ábila Cruzeiro/MG],[45:00 INTSão Paulo/SP 31 - Luiz de Araujo Gui...,[+01:00 2T3Rodrigo Caio Coquette Russo São Pau...,[]
6,Coritiba / PR,Atlético / GO,4 X 1,[[84Wilson Wilson Rodrigues de ... T(g)P15550...,[31:00 1T91NRHenrique Almeida Caixeta Nascente...,[18:00 1TCoritiba/PR 26 - Tomas Almino Bastos ...,"[41:00 1T5Alan Santos da Silva Coritiba/PR, 24...",[]
7,Grêmio / RS,Botafogo / RJ,2 X 0,"[[1M. Grohe Marcelo Grohe T(g)P168351, Grêmio ...","[45:00 1T17NRRamiro Moschen Benetti Grêmio/RS,...",[12:00 2TBotafogo/RJ 20 - Guilherme Augusto Vi...,[14:00 2T8Bruno Cesar Pereira Silva Botafogo/R...,[]
8,Bahia / BA,Athletico Paranaense / PR,6 X 2,"[[1Jean Jean Paulo Fernandes ... T(g)P420409, ...",[14:00 1T17NRGuilherme Milhomem Gusmao Atlétic...,[09:00 2TAtlético/PR 20 - Matheus Rossetto 33 ...,[23:00 2T45Marcos do Nascimento Teixeira Atlét...,[28:00 2T45Marcos do Nascimento Teixeira - Atl...
9,Ponte Preta / SP,Sport / PE,4 X 0,[[1Aranha Mario Lucio Duarte Costa T(g)P142720...,[39:00 1T9NRLucca Borges de Brito Ponte Preta/...,[45:00 INTSport/PE 37 - Edimar Ribeiro da Cost...,[41:00 1T5Rodrigo Vasconcelos Oliveira Sport/P...,[]


In [2]:
# Selecionar linhas onde 'Palmeiras / SP' aparece em 'Home' ou 'Away'
#df_transposto.loc[(df_transposto['Home'] == 'Palmeiras / SP') | (df_transposto['Away'] == 'Palmeiras / SP')]


In [3]:
df_squad.head(9)

,0,1,2,3,4,5,6,7,8
1,"{'Home': {'Squad': ['129292', '174360', '53542...","{'Home': {'Squad': ['129292', '174360', '53542...","{'Home': {'Squad': ['129292', '174360', '53542...","{'Home': {'Squad': ['129292', '174360', '15728...","{'Home': {'Squad': ['129292', '174360', '15728...","{'Home': {'Squad': ['129292', '174360', '15728...","{'Home': {'Squad': ['129292', '174360', '15728...",NaN,NaN
2,"{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...",NaN,NaN
3,"{'Home': {'Squad': ['427979', '186564', '16068...","{'Home': {'Squad': ['427979', '186564', '16068...","{'Home': {'Squad': ['427979', '186564', '16068...","{'Home': {'Squad': ['427979', '186564', '16068...","{'Home': {'Squad': ['427979', '186564', '16068...",NaN,NaN,NaN,NaN
4,"{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...",NaN
5,"{'Home': {'Squad': ['459582', '291542', '17241...","{'Home': {'Squad': ['459582', '172411', '61006...","{'Home': {'Squad': ['459582', '172411', '61006...","{'Home': {'Squad': ['459582', '172411', '61006...","{'Home': {'Squad': ['459582', '172411', '61006...","{'Home': {'Squad': ['459582', '172411', '61006...",NaN,NaN,NaN
6,"{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...",NaN,NaN,NaN
7,"{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...",NaN,NaN
8,"{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '308980', '33783...",NaN,NaN,NaN
9,"{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...",NaN,NaN


Teste inicial utilizando um jogo do brasileirão Série A 2014. Pegamos o primeiro jogo do Fluminense x Figueirense para criarmos a formatação das colunas e extração de informações da súmula.

In [76]:
players_teams = df_transposto.iloc[0, 3]
team_changes = df_transposto.iloc[0, 5]

start_players = df_squad.iloc[0,:]

# Criando DataFrame dos jogadores
new_df_players = pd.DataFrame(players_teams, columns=['player', 'team'])

def extract_id(player_string):
    # Ajuste para garantir que captura corretamente todas as variantes de ID
    match = re.search(r'T\(g\)?P(\d+)|RP(\d+)|TP(\d+)', player_string, re.IGNORECASE)
    if match:
        return next((m for m in match.groups() if m), None)
    return None

# Limpeza dos nomes dos jogadores
def clean_player_name(player):
    clean_name = re.sub(r'\s+T.*|\s+TP.*|\s+RP.*|\s+T(g).*', '', player)  # Remove texto após 'T' ou 'TP'
    return clean_name

new_df_players['player_id'] = new_df_players['player'].apply(extract_id)
new_df_players['player_name'] = new_df_players['player'].apply(clean_player_name)


def parse_team_changes(changes):
    pattern = re.compile(r'(\d{2}:\d{2}) (INT|\d+T)([\w\s]+/\w+) (\d+) - [^\d]+ (\d+) - [^\d]+')
    parsed_data = []
    for change in changes:
        match = pattern.search(change)
        if match:
            time, half, team, player_out_number, player_in_number = match.groups()
            team = team.strip()  # Limpeza para remover espaços extras
            parsed_data.append((time, half, team, player_out_number, player_in_number))
    return parsed_data


parsed_changes = parse_team_changes(team_changes)
print(parsed_changes[0])


new_df_players['Minutes Played'] = 90
new_df_players['Minute Entered'] = 0  # Todos os jogadores começam com entrada no minuto 0
new_df_players['Minute Exited'] = 90  # Todos os jogadores começam com saída no minuto 90

for time, half, team, player_out_number, player_in_number in parsed_changes:
    minute = int(time.split(':')[0])
    team = team.replace('/', ' / ')  # Formatando o nome do time corretamente
    
    if '2T' in half:
        minute_entered = 45 + minute
        minute_exited = 45 + minute
    else:
        minute_entered = minute
        minute_exited = minute

    # Atualizar o jogador que entrou
    mask_in = (new_df_players['player_name'].apply(lambda x: re.match(r'^' + player_in_number + r'\D', x) is not None) & 
               (new_df_players['team'] == team))
    new_df_players.loc[mask_in, 'Minute Entered'] = minute_entered
    new_df_players.loc[mask_in, 'Minute Exited'] = 90  # O jogador que entra fica até o final do jogo

    # Atualizar o jogador que saiu
    mask_out = (new_df_players['player_name'].apply(lambda x: re.match(r'^' + player_out_number + r'\D', x) is not None) & 
                (new_df_players['team'] == team))
    new_df_players.loc[mask_out, 'Minute Exited'] = minute_exited

# Recalcular os minutos jogados com base nos minutos de entrada e saída
new_df_players['Minutes Played'] = new_df_players['Minute Exited'] - new_df_players['Minute Entered']

#setando Home ou Away
home_team = df_transposto.iloc[0, 0]
new_df_players
new_df_players['status'] = np.where(new_df_players['team'] == home_team, 'Home', 'Away')

new_df_players = new_df_players.drop(columns='player')
# Visualizando o novo DataFrame
new_df_players

('17:00', '2T', 'Fluminense/RJ', '5', '70')


,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status
0,Fluminense / RJ,137990,12Diego Cava ... Diego Cavalieri,90,0,90,Home
1,Fluminense / RJ,178827,2Lucas Lucas Rios Marques,90,0,90,Home
2,Fluminense / RJ,180732,4Renato Cha ... Renato de Araujo Cha ...,90,0,90,Home
3,Fluminense / RJ,189708,9Henrique D ... Jose Henrique da Sil ...,90,0,90,Home
4,Fluminense / RJ,306146,11Wellington ... Wellington Alves da ...,26,64,90,Home
5,Fluminense / RJ,346294,15Léo Leonardo Pinheiro da ...,90,0,90,Home
6,Fluminense / RJ,565313,18Orejuela Jefferson Gabriel Or ...,90,0,90,Home
7,Fluminense / RJ,565315,20Sornoza Junior Nazareno Sorn ...,25,65,90,Home
8,Fluminense / RJ,161301,33Henrique Henrique Adriano Buss,90,0,90,Home
9,Fluminense / RJ,421523,37Wendel Marcus Wendel Valle ...,90,0,90,Home


In [100]:

def parse_goals(goals, home_team):
    pattern = re.compile(r'(\+?\d+)(:00)? (\d+T)([^\s]+)\s+([^\d]+?)\s+(.*)')

    
    parsed_goals = []
    for goal in goals:

        match = pattern.search(goal)

        if match:
            if '+' in (match.group(1)):
                minute = 45
            else:
                minute = int(match.group(1))

            half = match.group(3)

            scorer_info = match.group(4)
            print(match.group(5))
            player_name = match.group(5).strip()

            time_casa = df_transposto.iloc[0, 0]
            time_visitante = df_transposto.iloc[0, 1]

            time_casa = time_casa.replace(' / ', '/')
            time_visitante = time_visitante.replace(' / ', '/')

            if time_casa in  match.group(6):
                team = time_casa
            else: 
                team = time_visitante

            team = team.replace('/', ' / ')
            # Convertendo para minuto total do jogo
            if '2T' in half and minute != 45:  # Ajusta para o tempo total no caso do segundo tempo
                minute += 45
            
            # Verificar se é um gol contra (CT)
            if 'CT' in scorer_info:
                if team == home_team:
                    team_status = 'Away'
                else:
                    team_status = 'Home'
            else:
                if team == home_team:
                    team_status = 'Home'
                else:
                    team_status = 'Away'
            
            parsed_goals.append((minute, team_status))
    return parsed_goals

home_team = df_transposto.iloc[0, 0]
parsed_goals = parse_goals(df_transposto.iloc[0,4], home_team)

parsed_goals

Henrique
Ferraz
Nazareno
Javier
Henrique


[(3, 'Home'), (38, 'Away'), (57, 'Home'), (87, 'Away'), (45, 'Home')]

In [75]:
df_transposto.iloc[1,4]

['23:00 1T42NRMatheus Gonçalves Savio Flamengo/RJ',
 '13:00 2T8NRElias Mendes Trindade Atlético/MG']

In [7]:
df_transposto.iloc[8,6]

['41:00 1T5Rodrigo Vasconcelos Oliveira Sport/PE',
 '43:00 1T27Samuel Xavier Brito Sport/PE']

In [72]:

new_df_players['Goals For'] = 0

new_df_players['Goals Against'] = 0

for minute, team in parsed_goals:
    # Jogadores em campo no momento do gol
    mask = (new_df_players['Minute Entered'] <= minute) & (new_df_players['Minute Exited'] >= minute)
    # Incrementar gols a favor
    mask_for = mask & (new_df_players['status'] == team)
    new_df_players.loc[mask_for, 'Goals For'] += 1
    
    # Incrementar gols contra
    mask_against = mask & (new_df_players['status'] != team)
    new_df_players.loc[mask_against, 'Goals Against'] += 1

# Exibindo o DataFrame final para verificar os resultados
new_df_players

,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status,Goals For,Goals Against
0,Ponte Preta / SP,142720,1Aranha Mario Lucio Duarte Costa,90,0,90,Home,3,0
1,Ponte Preta / SP,183977,2Nino Paraíba Severino do Ramo Cle ...,90,0,90,Home,3,0
2,Ponte Preta / SP,299840,3Marllon Marllon Goncalves Je ...,90,0,90,Home,3,0
3,Ponte Preta / SP,167600,4Kadu Ricardo Martins de A ...,90,0,90,Home,3,0
4,Ponte Preta / SP,336832,5Naldo Marinaldo dos Santos ...,90,0,90,Home,3,0
5,Ponte Preta / SP,303237,6Joao Joao Lucas Cardoso,8,82,90,Home,0,0
6,Ponte Preta / SP,376734,7Clayson Clayson Henrique da ...,90,0,90,Home,3,0
7,Ponte Preta / SP,181803,8Elton Elton Junior Melo Ataide,90,0,90,Home,3,0
8,Ponte Preta / SP,292592,9Lucca Lucca Borges de Brito,90,0,90,Home,3,0
9,Ponte Preta / SP,443517,10Ravanelli Ravanelli Ferreira d ...,23,67,90,Home,1,0


In [9]:

def collect_unique_ids(data_folder, file_names, years):
    unique_ids = set()

    def process_dataframe(df):
        for column in df.columns:
            for cell in df[column]:
                if isinstance(cell, dict):  
                    for key in ['Home', 'Away']:
                        team_info = cell.get(key, {})
                        # Adicionar os IDs do 'Squad'
                        squad_ids = team_info.get('Squad', [])
                        unique_ids.update(squad_ids)


    for file_name in file_names:
        for year in years:
            file_path = os.path.join(data_folder, f'{file_name}_{year}_squads.json')
            if os.path.exists(file_path):
                df = pd.read_json(file_path)
                df_ = df.iloc[:,1]
                df = pd.DataFrame(df_)
                process_dataframe(df)
            else:
                print(f'Arquivo não encontrado: {file_path}')

    return list(unique_ids)

data_folder = '../Data/'
file_names = ['Serie_A']
years = range(2017, 2018)

unique_player_ids = collect_unique_ids(data_folder, file_names, years)

print(f'Total de IDs únicos coletados: {len(unique_player_ids)}')

Total de IDs únicos coletados: 28


In [10]:

df_players_final = new_df_players[new_df_players['player_id'].isin(unique_player_ids)]
df_players_final

,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status,Goals For,Goals Against
